In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import string
%matplotlib inline

In [ ]:
#importing dataset
messages = pd.read_csv("../input/spamraw.csv")
messages.head()
messages.describe()

In [ ]:
#Let's add a column of length to our data
messages['length'] = messages['text'].apply(len)
messages.head()

In [ ]:
#let's visualize
messages['length'].plot.hist(bins=50)
messages['length'].describe()

In [ ]:
#Creating histograms for Ham,Spam
messages.hist(column = 'length',by = 'type',bins = 60,figsize = (12,4))

**The real game starts from  here**

In [ ]:
#Creating Function
from nltk.corpus import stopwords
def text_process(text):
    """
    1.Remove punc
    2.Remove stop words
    3.Return list of clean text words
    """
    nonpunc = [char for char in text if char not in string.punctuation]
    nonpunc = ''.join(nonpunc)
    return[word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages['text'].head(5).apply(text_process)

Upto here the process is called Tokenization, which means converting a normal text string into a list of tokens.
Tokens are the just the word we actually want i.e.,** "Clean version of the words"**

Now, converting messages into Vectors step-by-step:

1)Count how many times does a word occurs in each message(known as Term Frequency)

2)Weight the counts, so that frequent tokens get lower weight(Inverse Document Frequency)

3)Normalize the vectors to unit length, to abstract from original text length

In [ ]:
#Step 1
from sklearn.feature_extraction.text import  CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_process).fit(messages['text'])
print(len(bow_transformer.vocabulary_)) #This is going to print total no. of vocab words

In [ ]:
#lets check transform vocabulary for one message
#bow is Bag of Words
mess4 = messages['text'][3]
print(mess4)
bow4 = bow_transformer.transform([mess4])
print(bow4)
print(bow4.shape)

In [ ]:
messages_bow = bow_transformer.transform(messages['text'])
print('shape of sparse matrix :',messages_bow.shape)
#Check amount of non-zero occurences
messages_bow.nnz

In [ ]:
#step2 & step3
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
#lets check 
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)


In [ ]:
#lets build our model
from sklearn.model_selection import train_test_split
msg_train,msg_test,type_train,type_test = train_test_split(messages['text'],
                                                             messages['type'],
                                                             test_size = 0.3)

Whatever we have done so far was just to know the basic idea of Stopwords,punctuations etc.But scikat learn has a in-built function called pipeline to do all such stuff. So, am using that in-built function now.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer = text_process)),
     ('tfidf',TfidfTransformer()),
      ('classifier',MultinomialNB())])

In [ ]:
#Now we can directly pass our message text and pipeline will do all our pre-processing for us
pipeline.fit(msg_train,type_train)
predictions = pipeline.predict(msg_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(type_test,predictions))

You can also use Random Forest by just changing in "Pipeline"